
# 任務1

In [ ]:
# @title 儲存格 1: 環境設定與套件安裝 (無變動)
# --- 環境設定與套件安裝 ---

# 確認 GPU 是否可用 (Colab 運行時 -> 變更執行階段類型 -> GPU)
import torch
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 是否可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 版本: {torch.version.cuda}")
    print(f"當前 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("警告: 未檢測到 GPU。訓練和推論將非常緩慢。")

# 安裝必要的套件
# - transformers: Hugging Face 模型庫
# - peft: Parameter-Efficient Fine-Tuning (用於 LoRA)
# - accelerate: 加速 Hugging Face 模型的訓練和推論
# - bitsandbytes: 用於 8-bit 量化，節省記憶體
# - datasets: 用於處理數據集
!pip install -q transformers peft accelerate bitsandbytes datasets librosa soundfile

# 確保 Dataset 類別被導入
from datasets import Dataset

# 掛載 Google Drive (推薦：將訓練成果和輸出檔案保存到 Drive)
# 如果你不使用 Google Drive，可以跳過這一步，但請確保相應的檔案路徑已修改為本地路徑。
from google.colab import drive
try:
    drive.mount('/content/drive')
    print("Google Drive 掛載成功。")
except Exception as e:
    print(f"Google Drive 掛載失敗: {e}。請手動掛載或修改檔案路徑。")

print("環境設定與套件安裝完成。")

PyTorch 版本: 2.6.0+cu124
CUDA 是否可用: True
CUDA 版本: 12.4
當前 GPU: Tesla T4
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive 掛載成功。
環境設定與套件安裝完成。


In [ ]:
# @title 儲存格 2: 定義輔助函式、檔案路徑、解壓縮與載入訓練數據
# --- 導入必要模組 ---
import os
import re
import json
import time # 導入 time 模組用於計時
import zipfile # 導入 zipfile 模組
from pathlib import Path # 導入 Path 模組
from datasets import Dataset # 導入 Dataset 類別

# --- 從 LLM 的輸出中提取 JSON 內容的輔助函式 (無變動) ---
def extract_json_from_output(generated_text):
    """
    使用正則表達式從 LLM 的輸出中提取 JSON 內容。
    會優先尋找 ```json 區塊，如果沒有找到，會嘗試尋找 {} 包裹的內容作為備用。
    """
    # 尋找 ```json 開頭和 ``` 結尾的區塊 - 讓正則更寬鬆處理空白和換行
    json_match = re.search(r'```json\s*(.*?)\s*```', generated_text, re.DOTALL)
    if json_match:
        json_content = json_match.group(1).strip()
        if json_content:
            try:
                json.loads(json_content)
                print("    (使用主要方法成功提取並驗證 JSON)")
                return json_content
            except json.JSONDecodeError:
                print(f"    (主要方法提取到內容但 JSON 無效: {json_content[:200]}...)")
                pass

    # 備用方案
    fallback_matches = re.findall(r'\{(.*?)\}', generated_text, re.DOTALL)
    for match_content in fallback_matches:
        fallback_content = '{' + match_content + '}'
        try:
            json.loads(fallback_content)
            print("    (使用備用方案提取到可能的 JSON)")
            return fallback_content
        except json.JSONDecodeError:
            pass

    print("    (未找到可提取的 JSON 內容)")
    return None

# --- 配置檔案路徑 ---
# 所有 AI CUP 相關的結果和模型都存放在這裡，方便管理
AICUP_RESULTS_BASE = "/content/drive/My Drive/AICUP_Results"
os.makedirs(AICUP_RESULTS_BASE, exist_ok=True)


# 微調後的 LLM 模型保存路徑
# 這個路徑將用於保存訓練好的 LoRA 適配器
FINE_TUNED_MODEL_OUTPUT_PATH = Path(AICUP_RESULTS_BASE) / "fine_tuned_llm_adapter"
os.makedirs(FINE_TUNED_MODEL_OUTPUT_PATH, exist_ok=True)
print(f"微調模型保存路徑: {FINE_TUNED_MODEL_OUTPUT_PATH}")

# Task 2 的輸出檔案路徑 (LLM 推論結果寫入這裡)
# 這個路徑仍然用於儲存 Task 2 的結果，但不再是本腳本的主要目標。
task2_output_path = Path(AICUP_RESULTS_BASE) / "task2_answer.txt"
print(f"Task 2 輸出路徑: {task2_output_path}")

# --- AI Cup 原始訓練數據集相關路徑 ---
zip_file_path_aicup = '/content/drive/My Drive/5月18AIcup.zip'
extract_folder_aicup = Path('/content/')
base_data_path_aicup = extract_folder_aicup / '5月18AIcup'

print("\n--- 開始處理 AI Cup 原始訓練數據集壓縮檔 ---")
if not base_data_path_aicup.exists():
    print(f"正在解壓縮 {zip_file_path_aicup} 到 {extract_folder_aicup}...")
    with zipfile.ZipFile(zip_file_path_aicup, 'r') as zip_ref:
        zip_ref.extractall(extract_folder_aicup)
    print("AI Cup 原始訓練數據集解壓縮完成。")
else:
    print(f"AI Cup 原始訓練資料集 '{base_data_path_aicup}' 已解壓縮，跳過。")


# 定義包含 task1_answer.txt 和音訊檔案的資料夾列表
# 用於 LLM 訓練的資料夾
training_subfolders = [
    "Training_Dataset_01",
    "Training_Dataset_02",
    "Validation_Dataset" # 依照您的描述，Validation_Dataset 也作為訓練集
]


# --- 新增函式：讀取並聚合 'task2_answer.txt' 格式的標註檔案 (無變動) ---
def load_and_aggregate_shi_labels(labels_file_path, audio_ids_in_dataset):
    """
    從 'task2_answer.txt' 格式的檔案中讀取敏感資訊標註，並將其聚合為每個 audio_id 的 JSON 格式。
    會篩選出只存在於 audio_ids_in_dataset 中的 ID。
    """
    aggregated_labels = {
        "AGE": {}, "TIME": {}, "DATE": {}, "DOCTOR": {}, "PROFESSION": {}
    }

    # 創建一個映射，將 task2_answer.txt 中的類別映射到 LLM 訓練預期的類別
    # 請檢查你的 task2_answer.txt 實際的類別名稱，並調整這裡的映射
    category_mapping = {
        "AGE": "AGE", "TIME": "TIME", "DATE": "DATE",
        "DOCTOR": "DOCTOR", "PROFESSION": "PROFESSION",
        "DURATION": "TIME",       # 將 DURATION 歸入 TIME
        "FAMILYNAME": "DOCTOR",   # 假設 FAMILYNAME 主要指醫生
        # 其他類別如果不需要 LLM 抽取，可以不放入 mapping，或映射到 "OTHER"
        "ID_NUMBER": "OTHER",
        "STREET": "OTHER",
        "CITY": "OTHER",
        "STATE": "OTHER",
        "ZIP": "OTHER",
        "ORGANIZATION": "OTHER",
        "PHONE": "OTHER",
        "EMAIL": "OTHER",
        "MEDICAL_RECORD": "OTHER",
        "PATIENT_ID": "OTHER",
        "HOSPITAL": "OTHER",
        "DEPARTMENT": "OTHER"
    }

    if not labels_file_path.exists():
        print(f"警告：標註文件 {labels_file_path} 未找到，將返回空標註。")
        return {}

    print(f"正在讀取標註文件: {labels_file_path}...")
    with open(labels_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # 假設 task2_answer.txt 的格式是：audio_id\tcategory\tstart\tend\ttext
            parts = line.strip().split('\t')
            # 這裡是最常出錯的地方，請根據您的 task2_answer.txt 實際的欄位數量和內容調整
            if len(parts) == 5: # 確保有足夠的欄位
                audio_id_str = parts[0]
                original_category = parts[1]
                shi_text = parts[4] # 敏感資訊文本在第五列

                # 確保只處理存在於當前數據集中的 audio_id
                if audio_id_str not in audio_ids_in_dataset:
                    continue

                mapped_category = category_mapping.get(original_category)

                # 只處理我們 LLM 預期輸出的五個核心類別
                if mapped_category and mapped_category in aggregated_labels:
                    if audio_id_str not in aggregated_labels[mapped_category]:
                        aggregated_labels[mapped_category][audio_id_str] = set()
                    aggregated_labels[mapped_category][audio_id_str].add(shi_text)
            else:
                # 警告不符合預期的行格式
                # print(f"警告: 標註文件 '{labels_file_path}' 中存在格式不符的行: {line.strip()[:100]}...")
                pass # 不打印過多警告，避免刷屏

    # 將 set 轉換回 list
    final_aggregated_labels = {}
    for cat, audio_ids_dict in aggregated_labels.items():
        for audio_id, texts_set in audio_ids_dict.items():
            if audio_id not in final_aggregated_labels:
                final_aggregated_labels[audio_id] = {
                    "AGE": [], "TIME": [], "DATE": [], "DOCTOR": [], "PROFESSION": []
                }
            final_aggregated_labels[audio_id][cat] = sorted(list(texts_set))

    return final_aggregated_labels


# 輔助函數：讀取並格式化 LLM 訓練數據集 (無變動)
def format_instruction_dataset_for_training(base_path, subfolders, aggregated_shi_labels):
    """
    從多個 task1_answer.txt 讀取數據，並構建 LLM 訓練所需的 (text, labels) 格式。
    此函數現在會使用預先聚合的敏感資訊標註。
    """
    all_training_samples = []

    # 首先，讀取所有的 transcription 並創建一個臨時字典
    transcriptions_temp = {}
    for folder_name in subfolders:
        file_path = base_path / folder_name / "task1_answer.txt"
        if not file_path.exists():
            print(f"警告：找不到 LLM 訓練文件 {file_path}，跳過此資料夾。")
            continue
        print(f"正在讀取 LLM 訓練數據: {file_path}...")
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t', 1)
                if len(parts) == 2:
                    audio_id_str = parts[0]
                    transcription = parts[1]
                    transcriptions_temp[audio_id_str] = transcription

    # 遍歷所有已讀取的 transcription，查找對應的敏感資訊標註
    for audio_id_str, transcription in transcriptions_temp.items():
        # 從聚合的標註中獲取當前 audio_id 的敏感資訊
        shi_for_this_audio = aggregated_shi_labels.get(audio_id_str, {
            "AGE": [], "TIME": [], "DATE": [], "DOCTOR": [], "PROFESSION": []
        })

        output_json_data = {
            "text": transcription,
            "AGE": shi_for_this_audio['AGE'],
            "TIME": shi_for_this_audio['TIME'],
            "DATE": shi_for_this_audio['DATE'],
            "DOCTOR": shi_for_this_audio['DOCTOR'],
            "PROFESSION": shi_for_this_audio['PROFESSION']
        }

        json_content_str = json.dumps(output_json_data, ensure_ascii=False, indent=4)
        labels_json_str = f"```json\n{json_content_str}\n```"

        all_training_samples.append({
            "text": transcription,
            "labels": labels_json_str
        })

    return Dataset.from_list(all_training_samples)


# --- 1. 載入訓練集的敏感資訊標註 ---
# 這個變數將儲存所有訓練集的 audio_id (從 task1_answer.txt 讀取)
training_audio_ids = set()
for folder_name in training_subfolders:
    task1_path = base_data_path_aicup / folder_name / "task1_answer.txt"
    if task1_path.exists():
        with open(task1_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t', 1)
                if len(parts) == 2:
                    training_audio_ids.add(parts[0])
print(f"訓練集包含 {len(training_audio_ids)} 個音訊 ID (從 task1_answer.txt)。")


# 載入所有訓練集的敏感資訊標註 (從 task2_answer.txt 讀取)
all_training_shi_labels = {}
for folder_name in training_subfolders:
    labels_file_path = base_data_path_aicup / folder_name / "task2_answer.txt"

    # 獲取當前資料夾的 audio_ids (為了 load_and_aggregate_shi_labels 篩選用)
    current_folder_audio_ids = set()
    task1_path = base_data_path_aicup / folder_name / "task1_answer.txt"
    if task1_path.exists():
        with open(task1_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t', 1)
                if len(parts) == 2:
                    current_folder_audio_ids.add(parts[0])

    aggregated_labels_for_folder = load_and_aggregate_shi_labels(labels_file_path, current_folder_audio_ids)
    all_training_shi_labels.update(aggregated_labels_for_folder) # 合併所有訓練集的標註

print(f"已從訓練集的標註文件中載入 {len(all_training_shi_labels)} 個音訊 ID 的敏感資訊標註。")
if len(all_training_shi_labels) == 0:
    print("警告: 未能從訓練集的 task2_answer.txt 載入任何敏感資訊標註。")
    print("請檢查 task2_answer.txt 的格式 (是否為 audio_id\\tcategory\\tstart\\tend\\ttext) ")
    print("以及 'load_and_aggregate_shi_labels' 函數中的 'category_mapping' 是否正確。")


# --- 2. 載入 LLM 訓練數據集 (轉錄 + 聚合後的敏感資訊標註) ---
print("正在載入和格式化 LLM 訓練數據集 (轉錄文本與敏感資訊標註合併)...")
train_dataset = format_instruction_dataset_for_training(base_data_path_aicup, training_subfolders, all_training_shi_labels)
print(f"LLM 訓練數據集載入完成。總共包含 {len(train_dataset)} 條數據。")

if len(train_dataset) > 0:
    print("LLM 訓練數據集範例 (第一條):")
    found_example_with_shi = False
    for i in range(len(train_dataset)):
        try:
            json_match = re.search(r'```json\s*(.*?)\s*```', train_dataset[i]['labels'], re.DOTALL)
            if json_match:
                labels_json = json.loads(json_match.group(1))
            else:
                labels_json = json.loads(train_dataset[i]['labels'])

            # 檢查是否有任何敏感資訊類別不為空
            if any(labels_json[cat] for cat in ["AGE", "TIME", "DATE", "DOCTOR", "PROFESSION"]):
                print(f"原始文本: {train_dataset[i]['text']}")
                print(f"期望標籤 (JSON): {train_dataset[i]['labels']}")
                found_example_with_shi = True
                break
        except json.JSONDecodeError as e:
            print(f"警告: 訓練數據中的 JSON 解析錯誤 (索引: {i}): {e}")
            print(f"錯誤的 JSON 字串片段: {train_dataset[i]['labels'][:500]}")
            continue

    if not found_example_with_shi:
        print("\n警告: 所有訓練數據集範例的敏感資訊標籤都為空或解析失敗。請檢查標註檔案和 category_mapping。")
        if len(train_dataset) > 0: # 至少打印第一個示例，即使它沒有 SHI
            print(f"原始文本 (第一條): {train_dataset[0]['text']}")
            print(f"期望標籤 (JSON) (第一條): {train_dataset[0]['labels']}")
else:
    print("警告: LLM 訓練數據集為空。請確認壓縮檔中的 task1_answer.txt 和標註文件是否包含內容。")


# --- 私有數據集 (Private dataset.zip) 解壓縮和路徑配置 ---
private_zip_file_path = Path('/content/drive/My Drive/private dataset/Private dataset.zip') # <-- 這裡已修改為 Path 物件
PRIVATE_DATASET_UNZIP_PATH = Path('/content/Private_Dataset_Temp') # 解壓縮到 Colab 臨時資料夾

# Task 1 輸出路徑 (用於 Private dataset)
TASK1_OUTPUT_DIR_FOR_PRIVATE = Path(AICUP_RESULTS_BASE) / "private_task1_answers"
os.makedirs(TASK1_OUTPUT_DIR_FOR_PRIVATE, exist_ok=True)
private_task1_output_file = TASK1_OUTPUT_DIR_FOR_PRIVATE / "task1_answer.txt"

print(f"\n--- 開始處理私有數據集壓縮檔 ---")
print(f"私有 Zip 檔案路徑: {private_zip_file_path}")
print(f"私有資料集解壓縮目標路徑: {PRIVATE_DATASET_UNZIP_PATH}")
print(f"私有資料集 Task 1 輸出路徑: {private_task1_output_file}")


if not PRIVATE_DATASET_UNZIP_PATH.exists() or not os.listdir(PRIVATE_DATASET_UNZIP_PATH):
    print(f"正在解壓縮 '{private_zip_file_path}' 到 '{PRIVATE_DATASET_UNZIP_PATH}'...")
    if not private_zip_file_path.exists():
        print(f"錯誤: 私有 Zip 檔案 '{private_zip_file_path}' 不存在。請檢查路徑。")
        raise FileNotFoundError(f"Private Zip file not found at {private_zip_file_path}")

    # 定義密碼
    password = "aicup20660205".encode('utf-8')

    try:
        with zipfile.ZipFile(private_zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(PRIVATE_DATASET_UNZIP_PATH, pwd=password)
        print("私有資料集解壓縮完成。")
    except RuntimeError as e:
        if "password required for extraction" in str(e) or "Bad password for file" in str(e):
            print(f"解壓縮失敗: {e}")
            print("錯誤：密碼不正確。請確認密碼 'aicup20660205' 是否正確無誤。")
            raise Exception(f"解壓縮私有資料集失敗: 密碼錯誤。")
        else:
            print(f"解壓縮私有資料集失敗: {e}")
            print("請檢查私有 Zip 檔案是否損壞或內容結構是否符合預期。")
            raise Exception(f"解壓縮私有資料集失敗: {e}")
    except Exception as e:
        print(f"解壓縮私有資料集失敗: {e}")
        print("請檢查私有 Zip 檔案是否損壞或內容結構是否符合預期。")
        raise Exception(f"解壓縮私有資料集失敗: {e}")
else:
    print(f"資料夾 '{PRIVATE_DATASET_UNZIP_PATH}' 已存在且非空，跳過解壓縮。")

# 驗證解壓縮後的內容（可選，用於確認）
print(f"\n=== 檢查 '{PRIVATE_DATASET_UNZIP_PATH}' 的內容 (前10個檔案/資料夾) ===")
if os.path.exists(PRIVATE_DATASET_UNZIP_PATH) and os.listdir(PRIVATE_DATASET_UNZIP_PATH):
    os.system(f'ls -l "{PRIVATE_DATASET_UNZIP_PATH}" | head -n 11')
else:
    print(f"警告: '{PRIVATE_DATASET_UNZIP_PATH}' 為空或不存在，解壓縮可能失敗。")

print("\n--- 檔案路徑、解壓縮與訓練數據載入完成 ---")

微調模型保存路徑: /content/drive/My Drive/AICUP_Results/fine_tuned_llm_adapter
Task 2 輸出路徑: /content/drive/My Drive/AICUP_Results/task2_answer.txt

--- 開始處理 AI Cup 原始訓練數據集壓縮檔 ---
AI Cup 原始訓練資料集 '/content/5月18AIcup' 已解壓縮，跳過。
訓練集包含 2314 個音訊 ID (從 task1_answer.txt)。
正在讀取標註文件: /content/5月18AIcup/Training_Dataset_01/task2_answer.txt...
正在讀取標註文件: /content/5月18AIcup/Training_Dataset_02/task2_answer.txt...
正在讀取標註文件: /content/5月18AIcup/Validation_Dataset/task2_answer.txt...
已從訓練集的標註文件中載入 1237 個音訊 ID 的敏感資訊標註。
正在載入和格式化 LLM 訓練數據集 (轉錄文本與敏感資訊標註合併)...
正在讀取 LLM 訓練數據: /content/5月18AIcup/Training_Dataset_01/task1_answer.txt...
正在讀取 LLM 訓練數據: /content/5月18AIcup/Training_Dataset_02/task1_answer.txt...
正在讀取 LLM 訓練數據: /content/5月18AIcup/Validation_Dataset/task1_answer.txt...
LLM 訓練數據集載入完成。總共包含 2314 條數據。
LLM 訓練數據集範例 (第一條):
原始文本: Yeah, I imagine it would — sorry, go ahead. So it's supposed to work immediately, right? Yep. So we'll see if I'm productive tomorrow. I hope I'm productive today. I've actually been trying 

In [ ]:
# @title 儲存格 3: LLM 模型微調 (訓練功能)
# --- LLM 模型微調 (訓練功能) ---

# 導入訓練所需的函式庫
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import DataCollatorForLanguageModeling

# ==============================================================================
# 請注意：為了確保 trl 庫的 SFTTrainer 可用，雖然你目前的程式碼
# 使用了 transformers.Trainer，但為了未來可能的擴展或調試，
# 確保相關的函式庫已被安裝仍然很重要。
# 如果你之前有遇到 'trl' 錯誤，請確保在某個地方執行過安裝：
# !pip install -q trl bitsandbytes accelerate
# ==============================================================================

print("--- 開始 LLM 模型訓練準備 ---")

# --- 1. 載入基礎 LLM 模型和分詞器 ---
# 這裡使用你的原始 LLM 模型名稱作為微調的基礎。
BASE_LLM_MODEL_NAME = "facebook/opt-1.3b"

print(f"正在載入基礎 LLM 模型 ({BASE_LLM_MODEL_NAME}) 和分詞器...")
tokenizer = AutoTokenizer.from_pretrained(BASE_LLM_MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Tokenizer 的 pad_token 已設置為 eos_token: {tokenizer.eos_token}")

model = AutoModelForCausalLM.from_pretrained(
    BASE_LLM_MODEL_NAME,
    load_in_8bit=True, # 使用 8-bit 量化節省記憶體
    torch_dtype=torch.float16, # 使用 float16 減少記憶體消耗並加速計算
    device_map="auto",
)
model.config.use_cache = False # 在訓練時禁用 use_cache 以避免警告和記憶體問題

print("基礎 LLM 模型和分詞器載入完成。")

# --- 2. 配置 LoRA (Low-Rank Adaptation) ---
lora_config = LoraConfig(
    r=16, # LoRA 秩，可影響模型表達能力和訓練參數數量。保持原始設定。
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"], # 針對注意力層的 q_proj 和 v_proj。可以考慮加入 "k_proj" 但會略增參數。
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model) # 為 k-bit 訓練準備模型
model = get_peft_model(model, lora_config) # 應用 LoRA 適配器

print("LoRA 配置完成，模型已準備好進行 PEFT 微調。")
model.print_trainable_parameters()

# --- 3. 定義資料處理函式 ---
def tokenize_function(examples):
    full_text = []
    for text_input, json_label_output in zip(examples["text"], examples["labels"]):
        prompt_template = f'''You are an expert in extracting sensitive health information.
Strictly follow the output format.
Extract Sensitive Health Information (SHI) from the following text.
The text is a transcription of a doctor-patient conversation.
Identify and categorize the SHI into the following specific categories:
- AGE: Any mention of age or age range (e.g., "69-year-old", "in his 50s").
- TIME: Specific times of day, durations mentioned as time (e.g., "morning", "3 PM", "two hours").
- DATE: Specific dates, days of the week, months, or years related to events (e.g., "August 12, 1999", "last Monday", "in 2023").
- DOCTOR: Names of doctors or medical practitioners.
- PROFESSION: Any mention of a profession or occupation related to healthcare or the patient (e.g., "nurse", "psychiatrist", "engineer").

Return ONLY the extracted SHI in a single JSON object. The JSON object MUST be enclosed within ```json and ``` blocks. Do NOT include any introductory text, conversational phrases, explanations, or other code blocks (like ```python, ```javascript, etc.) before or after the JSON block. The JSON object must strictly follow this structure:
```json
{{
    "text": "{text_input}",
    "AGE": ["list all extracted AGE here"],
    "TIME": ["list all extracted TIME here"],
    "DATE": ["list all extracted DATE here"],
    "DOCTOR": ["list all extracted DOCTOR names here"],
    "PROFESSION": ["list all extracted PROFESSION here"]
}}
If a category has no corresponding information found in the text, its list in the JSON should be empty [].
Extract ONLY information that is explicitly mentioned in the text.
'''
        full_text.append(prompt_template + json_label_output)

    # === 加速點 1: 減少 max_length ===
    # 2500 是一個非常大的序列長度，會導致大量填充和計算。
    # 降低到 1024 通常足以覆蓋大部分文本，並顯著加快訓練速度。
    # 如果您的數據集文本平均長度很短，甚至可以進一步降低到 512。
    MAX_SEQUENCE_LENGTH = 1024 # 從 2500 降低到 1024

    tokenized_inputs = tokenizer(
        full_text,
        max_length=MAX_SEQUENCE_LENGTH, # 使用新的較小值
        truncation=True,
        padding="max_length"
    )
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

# --- 針對快速驗證的數據集調整 ---
QUICK_TEST_DATA_LIMIT = None # 預設使用所有數據，但將 epochs 設為 1

if QUICK_TEST_DATA_LIMIT is not None and QUICK_TEST_DATA_LIMIT < len(train_dataset):
    print(f"注意：為快速驗證，訓練數據集將限制為前 {QUICK_TEST_DATA_LIMIT} 個樣本。")
    limited_train_dataset = train_dataset.select(range(QUICK_TEST_DATA_LIMIT))
else:
    limited_train_dataset = train_dataset
    print("使用所有訓練數據。")

# 確保在 map 函數中使用 os.cpu_count() 來加速數據處理（如果環境允許）
import os
num_processing_workers = os.cpu_count() if os.cpu_count() else 1 # 如果獲取不到 CPU 數量，預設為 1

tokenized_train_dataset = limited_train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text", "labels"],
    num_proc=num_processing_workers # 使用多進程加速分詞
)
print("訓練資料集已分詞。")
print("分詞後的訓練數據集範例 (第一條):")
print(tokenized_train_dataset[0])


# --- 4. 定義訓練參數 ---
training_args = TrainingArguments(
    output_dir=FINE_TUNED_MODEL_OUTPUT_PATH,
    num_train_epochs=1, # 訓練 Epoch 數保持 1
    # === 加速點 2: 調整批次大小和梯度累積步數 ===
    # 從 per_device_train_batch_size=2, gradient_accumulation_steps=4 (有效批次=8)
    # 調整為 per_device_train_batch_size=4, gradient_accumulation_steps=2 (有效批次=8)
    # 這樣每次 GPU 處理更多樣本，可能提高 GPU 利用率。
    per_device_train_batch_size=4, # <--- 增加實際批次大小
    gradient_accumulation_steps=2, # <--- 減少梯度累積步數以維持有效批次大小
    # 或者，如果想更激進地加速（可能增加 OOM 風險），可以嘗試：
    # per_device_train_batch_size=4,
    # gradient_accumulation_steps=4, # 有效批次大小為 16
    gradient_checkpointing=True, # 啟用梯度檢查點以節省記憶體，這點很有用
    learning_rate=2e-4,
    optim="paged_adamw_8bit", # 使用 paged_adamw_8bit 優化器，這是最佳選擇
    logging_steps=10, # 每 10 步記錄一次日誌
    save_strategy="epoch", # 訓練結束時保存模型
    save_total_limit=1,
    report_to="none",
    remove_unused_columns=False,
    # 你的原始程式碼已經有 load_in_8bit 和 torch_dtype=torch.float16，
    # 這裡的 TrainingArguments 不需要額外的 fp16=True，因為模型已經是 float16。
)

print("訓練參數已設置。")

# --- 5. 創建 Trainer 並開始訓練 ---
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=training_args,
    data_collator=data_collator,
)

print("--- 開始訓練 LLM 模型 (這應該會更快) ---")
trainer.train()
print("LLM 模型訓練完成！")

# --- 6. 保存微調後的 LoRA 適配器 ---
trainer.model.save_pretrained(FINE_TUNED_MODEL_OUTPUT_PATH)
tokenizer.save_pretrained(FINE_TUNED_MODEL_OUTPUT_PATH)

print(f"微調後的 LoRA 適配器和 Tokenizer 已保存到: {FINE_TUNED_MODEL_OUTPUT_PATH}")
print("--- LLM 模型訓練結束 ---")

--- 開始 LLM 模型訓練準備 ---
正在載入基礎 LLM 模型 (facebook/opt-1.3b) 和分詞器...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


基礎 LLM 模型和分詞器載入完成。
LoRA 配置完成，模型已準備好進行 PEFT 微調。
trainable params: 3,145,728 || all params: 1,318,903,808 || trainable%: 0.2385
使用所有訓練數據。


Map (num_proc=2):   0%|          | 0/2314 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


訓練資料集已分詞。
分詞後的訓練數據集範例 (第一條):
{'labels': [2, 1185, 32, 41, 3827, 11, 37213, 5685, 474, 335, 4, 50118, 5320, 12127, 352, 1407, 5, 4195, 7390, 4, 50118, 40884, 15664, 12274, 15589, 1309, 3522, 36, 10237, 100, 43, 31, 5, 511, 2788, 4, 50118, 133, 2788, 16, 10, 37118, 9, 10, 3299, 12, 23846, 1607, 4, 50118, 46692, 4591, 8, 18072, 2072, 5, 4584, 100, 88, 5, 511, 2167, 6363, 35, 50118, 12, 83, 8800, 35, 5053, 4521, 9, 1046, 50, 1046, 1186, 36, 242, 4, 571, 482, 22, 4563, 12, 180, 12, 279, 1297, 22, 179, 39, 654, 29, 18653, 50118, 12, 18034, 35, 37699, 498, 9, 183, 6, 385, 42936, 2801, 25, 86, 36, 242, 4, 571, 482, 22, 24863, 1297, 22, 246, 2784, 1297, 22, 7109, 722, 18653, 50118, 12, 211, 8625, 35, 37699, 5461, 6, 360, 9, 5, 186, 6, 377, 6, 50, 107, 1330, 7, 1061, 36, 242, 4, 571, 482, 22, 20041, 316, 6, 6193, 1297, 22, 13751, 302, 1297, 22, 179, 291, 1922, 18653, 50118, 12, 14010, 7164, 3411, 35, 34557, 9, 3333, 50, 1131, 21492, 4, 50118, 12, 4729, 21600, 43400, 35, 5053, 4521, 9, 10, 12409,

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,1.622300
20,1.093300
30,0.723900
40,0.547300
50,0.474000
60,0.426000
70,0.431700
80,0.416200
90,0.400300
100,0.397300


Step,Training Loss
10,1.622300
20,1.093300
30,0.723900
40,0.547300
50,0.474000
60,0.426000
70,0.431700
80,0.416200
90,0.400300
100,0.397300


LLM 模型訓練完成！
微調後的 LoRA 適配器和 Tokenizer 已保存到: /content/drive/My Drive/AICUP_Results/fine_tuned_llm_adapter
--- LLM 模型訓練結束 ---


In [ ]:
# @title 儲存格 4: Whisper 模型載入 (用於 Task 1 推論) - LLM 模型已被移除以節省資源
# --- Whisper 模型載入 (用於 Task 1 推論) ---

# 導入推論所需的函式庫
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import torch

print("--- 開始載入 Whisper 模型 (用於 Task 1 推論) ---")

# --- 1. Whisper 模型載入 (直接載入 Processor 和 Model) ---
# 這裡載入一個預訓練的 Whisper 模型，用於語音轉文字
# !!! 你可以根據需求調整 Whisper 模型大小 (例如 "small", "base", "medium", "large") !!!
# 如果時間仍然超限或記憶體不足，請考慮使用 "openai/whisper-base" 或 "openai/whisper-small"
WHISPER_MODEL_NAME = "openai/whisper-medium" # <-- 替換為你想要的 Whisper 模型大小！

print(f"正在載入 Whisper 處理器 ({WHISPER_MODEL_NAME})...")
processor = AutoProcessor.from_pretrained(WHISPER_MODEL_NAME)

print(f"正在載入 Whisper 模型 ({WHISPER_MODEL_NAME})...")
try:
    # 嘗試將模型載入到 GPU
    whisper_model_for_transcription = AutoModelForSpeechSeq2Seq.from_pretrained(
        WHISPER_MODEL_NAME,
        torch_dtype=torch.float16, # 使用 float16 減少記憶體消耗
        device_map="auto", # 自動將模型載入到可用設備 (GPU)
    )
    print("Whisper 模型載入完成 (使用 auto device_map)。")
except Exception as e:
    print(f"載入 Whisper 模型到 GPU 時發生錯誤: {e}")
    # Fallback to CPU if auto fails, or explicitly load to CPU
    print("回退到 CPU 載入 Whisper 模型，這會比較慢。")
    whisper_model_for_transcription = AutoModelForSpeechSeq2Seq.from_pretrained(
        WHISPER_MODEL_NAME,
        torch_dtype=torch.float16, # 即使在 CPU 也保持 float16
        device_map="cpu", # 強制使用 CPU
    )
    print("Whisper 模型載入完成 (已回退到 CPU)。")

# 將模型設置為評估模式
whisper_model_for_transcription.eval()

print("--- Whisper 模型載入完成 ---")

--- 開始載入 Whisper 模型 (用於 Task 1 推論) ---
正在載入 Whisper 處理器 (openai/whisper-medium)...
正在載入 Whisper 模型 (openai/whisper-medium)...
Whisper 模型載入完成 (使用 auto device_map)。
--- Whisper 模型載入完成 ---


In [ ]:
# @title 儲存格 5: 對私有資料集執行 Task 1 語音轉錄
# --- 對私有資料集執行 Task 1 語音轉錄 ---

import os
import time
import librosa
import soundfile as sf
import numpy as np
import gc # 導入垃圾回收模組
import torch # 確保 torch 已經導入

# ... (前面代碼保持不變，包括路徑變數等) ...

# 以追加模式 ('a') 打開 Task 1 輸出檔案
with open(private_task1_output_file, "a", encoding="utf-8") as task1_file_append:
    start_time = time.time() # 開始計時
    processed_this_run = 0 # 計數本次運行成功處理的檔案

    for i, audio_path_obj in enumerate(audio_files):
        audio_id = audio_path_obj.stem # 獲取檔案名作為 audio_id
        audio_path_str = str(audio_path_obj) # 轉換為字串路徑

        if audio_id in processed_ids_task1:
            continue # 跳過已處理的檔案

        print(f"\n--- 正在處理檔案 ID: {audio_id} ({i+1}/{len(audio_files)}) --- (Task 1)")

        try:
            # 載入音訊檔案 (此部分保持不變)
            audio_input = None
            target_sample_rate = 16000 # Whisper 期望 16kHz

            try:
                print(f"    嘗試使用 librosa 載入音訊: {audio_path_str}")
                audio_input, current_sample_rate = librosa.load(audio_path_str, sr=None, dtype='float32')

                if current_sample_rate != target_sample_rate:
                    print(f"    librosa 載入後採樣率為 {current_sample_rate}，重新取樣至 {target_sample_rate}Hz...")
                    audio_input = librosa.resample(y=audio_input, orig_sr=current_sample_rate, target_sr=target_sample_rate)
                sample_rate = target_sample_rate

                if audio_input is None or audio_input.size == 0 or np.sum(np.abs(audio_input)) == 0:
                    raise ValueError(f"載入的音訊檔案 ({audio_path_str}) 為空或全為零。")

            except Exception as e_librosa:
                print(f"    Librosa 載入音訊 ({audio_path_str}) 時發生錯誤: {e_librosa}。嘗試使用 soundfile 備用方案。")
                try:
                    audio_input, current_sample_rate = sf.read(audio_path_str, dtype='float32')
                    if audio_input.ndim > 1:
                        audio_input = audio_input.mean(axis=1)

                    if current_sample_rate != target_sample_rate:
                        print(f"    soundfile 載入後採樣率為 {current_sample_rate}，重新取樣至 {target_sample_rate}Hz...")
                        audio_input = librosa.resample(y=audio_input, orig_sr=current_sample_rate, target_sr=target_sample_rate)
                        sample_rate = target_sample_rate
                    else:
                        sample_rate = current_sample_rate

                    if audio_input is None or audio_input.size == 0 or np.sum(np.abs(audio_input)) == 0:
                        raise ValueError(f"Soundfile 載入的音訊檔案 ({audio_path_str}) 為空或全為零。")

                except Exception as e_sf:
                    print(f"    Soundfile 載入音訊 ({audio_path_str}) 時也發生錯誤: {e_sf}。跳過此檔案。")
                    continue

            # === 關鍵修改點：更強制的 mel feature 填充 AND 數據類型轉換 ===
            # 獲取原始的 mel features (不進行任何填充或截斷)
            raw_input_features = processor.feature_extractor(
                audio_input, sampling_rate=sample_rate, return_tensors="pt"
            ).input_features

            expected_mel_length = 3000 # Whisper 期望的 mel feature 長度
            current_mel_length = raw_input_features.shape[-1]

            if current_mel_length < expected_mel_length:
                # 計算需要填充的長度
                padding_needed = expected_mel_length - current_mel_length
                # 使用 PyTorch 的 pad 函數在最後一個維度進行零填充
                padded_input_features = torch.nn.functional.pad(raw_input_features, (0, padding_needed))
            elif current_mel_length > expected_mel_length:
                # 如果因為某些原因超過了，則截斷。
                padded_input_features = raw_input_features[..., :expected_mel_length]
            else:
                # 長度已經符合，無需填充
                padded_input_features = raw_input_features

            # ====== 新增的資料類型轉換部分 ======
            # 將填充後的特徵轉換為 torch.float16 (半精度浮點數)
            padded_input_features = padded_input_features.to(torch.float16)
            # ==================================

            inputs = {"input_features": padded_input_features} # 構建最終的 inputs dictionary

            # 將輸入張量移動到 GPU (如果可用)
            if torch.cuda.is_available():
                inputs = {k: v.to("cuda") for k, v in inputs.items()}
            else:
                inputs = {k: v.to("cpu") for k, v in inputs.items()} # 確保在 CPU 上也能運行

            # 執行推論
            with torch.no_grad():
                generated_ids = whisper_model_for_transcription.generate(inputs["input_features"])

            # 解碼生成 ID 為文本
            transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
            print(f"    轉錄結果: {transcription}")

            # 將結果寫入 Task 1 輸出檔案
            task1_file_append.write(f"{audio_id}\t{transcription}\n")
            task1_file_append.flush() # 確保每處理一個檔案就寫入磁碟

            processed_this_run += 1
            processed_ids_task1.add(audio_id) # 更新已處理 ID 集合

            # 每次處理完畢後手動進行垃圾回收並清空 CUDA 快取
            del inputs, generated_ids, audio_input
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect() # 強制進行垃圾回收
            print("    已清除記憶體快取。")


        except Exception as e:
            print(f"  處理檔案 ID: {audio_id} 時發生未預期的錯誤 (Task 1): {e}")
            # 如果處理單個檔案 Task 1 時出錯，記錄錯誤並繼續處理下一個檔案
            pass # 使用 pass 跳過錯誤處理，繼續迴圈

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"\n--- Task 1 轉錄處理完成總結 ---")
    print(f"本次運行成功處理了 {processed_this_run} 個音訊檔案的 Task 1 轉錄。")
    print(f"所有音訊檔案的總處理時間: {elapsed_time:.2f} 秒 ({elapsed_time / 60:.2f} 分鐘)。")
    print(f"Task 1 轉錄結果已保存到: {private_task1_output_file}")

print("=== 私有資料集 Task 1 語音轉錄處理完成 ===")


--- 正在處理檔案 ID: 60014 (1/710) --- (Task 1)
    嘗試使用 librosa 載入音訊: /content/Private_Dataset_Temp/private/60014.wav


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


    轉錄結果:  were met. Specific dates such as July 19, 2062 and July 28, 2062 highlight significant interactions in her medical timeline. Amanda's story reflects the dedication and teamwork of medical professionals who prioritize patient well-being. Her records stand as a testament to the importance of coordinated and meticulous health care.
    已清除記憶體快取。

--- 正在處理檔案 ID: 60015 (2/710) --- (Task 1)
    嘗試使用 librosa 載入音訊: /content/Private_Dataset_Temp/private/60015.wav
    轉錄結果:  reports that Dr. T once performed the analysis on June 6, 2063. The final HER2-CISH analysis was completed on June 6, 2063.
    已清除記憶體快取。

--- 正在處理檔案 ID: 60018 (3/710) --- (Task 1)
    嘗試使用 librosa 載入音訊: /content/Private_Dataset_Temp/private/60018.wav
    轉錄結果:  and medical conditions.
    已清除記憶體快取。

--- 正在處理檔案 ID: 60022 (4/710) --- (Task 1)
    嘗試使用 librosa 載入音訊: /content/Private_Dataset_Temp/private/60022.wav
    轉錄結果:  Here, Catherine was attended to by the skilled hands of Dr. Marce, ensuring her health concer